<a href="https://colab.research.google.com/github/JOEL352006/Pdf-Chat-With-Gemini/blob/main/PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai PyMuPDF streamlit pyngrok


In [ ]:
from google.colab import files
uploaded = files.upload()

pdf_path = next(iter(uploaded))
print("Uploaded PDF:", pdf_path)


In [ ]:
%%writefile app.py
import streamlit as st
import fitz
import google.generativeai as genai

GOOGLE_API_KEY = "YOUR_GEMINI_API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)

def extract_text_chunks(pdf_file):
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    chunks = []
    for page in doc:
        text = page.get_text().strip()
        if text:
            chunks.append(text)
    return chunks

def find_relevant_chunk(chunks, query):
    model = genai.GenerativeModel('models/gemini-1.5-flash')
    best_chunk = ""
    best_score = -1

    for chunk in chunks:
        prompt = f"""
Rate the relevance (0 to 10) of this content to the question: "{query}"
Content:
\"\"\"
{chunk}
\"\"\"
Respond only with a number.
"""
        try:
            response = model.generate_content(prompt)
            score = int(''.join(filter(str.isdigit, response.text)))
        except:
            score = 0

        if score > best_score:
            best_score = score
            best_chunk = chunk

    return best_chunk

st.set_page_config(page_title="PDF Q/A Chatbot - Gemini", layout="wide")
st.title("📄 Gemini PDF Chatbot (Content-Strict)")

uploaded_pdf = st.file_uploader("Upload a PDF file", type="pdf")

if uploaded_pdf is not None:
    chunks = extract_text_chunks(uploaded_pdf)
    st.success(f"✅ PDF loaded with {len(chunks)} pages.")

    user_query = st.text_input("Ask a question (answer will come only from PDF):")

    if user_query:
        relevant_chunk = find_relevant_chunk(chunks, user_query)
        model = genai.GenerativeModel('models/gemini-1.5-flash')

        prompt = f"""
You must answer strictly from the following PDF content only.
If the answer is not found in this content, reply:
"I cannot answer this question as it is not present in the provided document."

PDF Content:
\"\"\"
{relevant_chunk}
\"\"\"

Question: {user_query}
"""

        response = model.generate_content(prompt)
        st.write("🧠 Gemini says:", response.text)


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501
